In [1]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

 
df = pd.read_csv("crime.csv") 
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [3]:

df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str


df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()
df

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,DATE,TIME,day_of_week
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016-06-15,11:31:00 PM,Wednesday
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2017-10-11,12:30:00 PM,Wednesday
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016-03-04,8:00:00 PM,Friday
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018-01-30,7:20:00 PM,Tuesday
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017-06-22,8:53:00 PM,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526095,2019688353,2019688353121200,1212,0,robbery-car-jacking,robbery,10/27/2019 11:45:00 PM,NaN,10/28/2019 12:08:00 AM,E COLFAX AVE / N XENIA ST,...,-104.885821,39.740155,2,223,east-colfax,1,0,2019-10-27,11:45:00 PM,Sunday
526096,2019688368,2019688368540100,5401,0,traffic-accident-hit-and-run,traffic-accident,10/28/2019 12:20:00 AM,NaN,10/28/2019 12:35:00 AM,1959 S BROADWAY ST,...,-104.987794,39.681001,3,313,overland,0,1,2019-10-28,12:20:00 AM,Monday
526097,2019688404,2019688404131302,1313,2,assault-dv,other-crimes-against-persons,10/28/2019 1:13:00 AM,NaN,10/28/2019 3:26:00 AM,7030 N TOWER RD,...,-104.770112,39.824869,5,522,dia,1,0,2019-10-28,1:13:00 AM,Monday
526098,2019688441,2019688441131302,1313,2,assault-dv,other-crimes-against-persons,10/28/2019 2:10:00 AM,NaN,10/28/2019 4:25:00 AM,3831 N SHOSHONE ST,...,-105.009777,39.769921,1,111,sunnyside,1,0,2019-10-28,2:10:00 AM,Monday


In [4]:
df_WCC = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID']]
df_WCC = df_WCC.loc[df_WCC.OFFENSE_CATEGORY_ID == 'white-collar-crime']
df_WCC_am =  df_WCC.loc[df_WCC.TIME.str[-2:] == "AM" ]
df_WCC_am['TIME'] = [x[:-3] for x in df_WCC_am['TIME']]
df_WCC_am['TIME'] =  pd.to_datetime(df_WCC_am['TIME'], format='%H:%M:%S')
# df_WCC_am['TIME'] = [d.time() for d in df_WCC_am['TIME']]
df_WCC_am

/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID
629,white-collar-crime,1900-01-01 01:00:00,Tuesday,dia
712,white-collar-crime,1900-01-01 12:00:00,Thursday,capitol-hill
730,white-collar-crime,1900-01-01 08:50:00,Thursday,five-points
814,white-collar-crime,1900-01-01 12:00:00,Wednesday,harvey-park
822,white-collar-crime,1900-01-01 12:00:00,Sunday,chaffee-park
...,...,...,...,...
525240,white-collar-crime,1900-01-01 08:30:00,Thursday,skyland
525523,white-collar-crime,1900-01-01 12:00:00,Tuesday,five-points
525657,white-collar-crime,1900-01-01 12:00:00,Wednesday,congress-park
525733,white-collar-crime,1900-01-01 01:00:00,Sunday,baker


In [5]:
df_WCC = df_WCC[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID']]
df_WCC = df_WCC.loc[df_WCC.OFFENSE_CATEGORY_ID == 'white-collar-crime']
df_WCC_pm =  df_WCC.loc[df_WCC.TIME.str[-2:] == "PM" ]
df_WCC_pm['TIME'] = [x[:-3] for x in df_WCC_pm['TIME']]
df_WCC_pm['TIME'] =  pd.to_datetime(df_WCC_pm['TIME'], format='%H:%M:%S')
df_WCC_pm['TIME'] = df_WCC_pm['TIME'] + timedelta(hours=12)
# df_WCC_pm['TIME'] = [d.time() for d in df_WCC_pm['TIME']]
df_WCC_pm

/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/missybueno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID
18,white-collar-crime,1900-01-02 00:00:00,Thursday,washington-park-west
25,white-collar-crime,1900-01-02 00:00:00,Monday,hampden-south
248,white-collar-crime,1900-01-02 00:00:00,Tuesday,capitol-hill
289,white-collar-crime,1900-01-01 14:00:00,Wednesday,stapleton
366,white-collar-crime,1900-01-01 18:00:00,Monday,westwood
...,...,...,...,...
525661,white-collar-crime,1900-01-01 13:00:00,Thursday,north-capitol-hill
525679,white-collar-crime,1900-01-01 14:07:00,Thursday,lincoln-park
525849,white-collar-crime,1900-01-01 17:00:00,Monday,dia
525858,white-collar-crime,1900-01-01 13:15:00,Thursday,lincoln-park


In [6]:
final_WCC = pd.concat([df_WCC_am, df_WCC_pm], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [7]:
final_WCC.loc[final_WCC['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object1.time())&(final_WCC['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object2.time())&(final_WCC['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object3.time())&(final_WCC['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object4.time())&(final_WCC['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object5.time())&(final_WCC['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_WCC['TIME'] = [d.time() for d in final_WCC['TIME']]
final_WCC

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID
0,white-collar-crime,04:00:00,Tuesday,dia
1,white-collar-crime,12:00:00,Thursday,capitol-hill
2,white-collar-crime,12:00:00,Thursday,five-points
3,white-collar-crime,12:00:00,Wednesday,harvey-park
4,white-collar-crime,12:00:00,Sunday,chaffee-park
...,...,...,...,...
7201,white-collar-crime,16:00:00,Thursday,north-capitol-hill
7202,white-collar-crime,16:00:00,Thursday,lincoln-park
7203,white-collar-crime,20:00:00,Monday,dia
7204,white-collar-crime,16:00:00,Thursday,lincoln-park


In [8]:
WCC_csv = final_WCC[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
WCC_csv = WCC_csv.sample(5000)
WCC_csv.to_csv('WCC.csv')


!python apriori.py -f 'WCC.csv' -s 0.0012

item: ('Monday', 'jefferson-park') , 0.001
item: ('harvey-park-south', 'Thursday') , 0.001
item: ('east-colfax', '23:59:00') , 0.001
item: ('city-park', 'Tuesday') , 0.001
item: ('civic-center', 'Friday') , 0.001
item: ('west-highland', 'Thursday') , 0.001
item: ('montbello', '23:59:00') , 0.001
item: ('cheesman-park', 'Tuesday') , 0.001
item: ('cory-merrill', 'Friday') , 0.001
item: ('congress-park', 'Wednesday') , 0.001
item: ('south-park-hill', 'Monday') , 0.001
item: ('college-view-south-platte', 'Friday') , 0.001
item: ('Wednesday', 'elyria-swansea') , 0.001
item: ('04:00:00', 'athmar-park') , 0.001
item: ('16:00:00', 'regis') , 0.001
item: ('ruby-hill', 'Monday') , 0.001
item: ('04:00:00', 'fort-logan') , 0.001
item: ('20:00:00', 'bear-valley') , 0.001
item: ('12:00:00', 'fort-logan') , 0.001
item: ('20:00:00', 'southmoor-park') , 0.001
item: ('hale', '20:00:00') , 0.001
item: ('cory-merrill', 'Monday') , 0.001
item: ('globeville', 'Wednesday') , 0.001
item: ('mar-lee', '08:00:00